# Emotion Detection

**Importing Libraries**

In [ ]:
import matplotlib.pyplot as plt
from glob import glob
import cv2
import random
import os
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Input,Dropout,Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization,Activation,MaxPooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG,Image
import tensorflow as tf
print("TensorFlow Version:",tf.__version__)

**Exploring Dataset**

In [ ]:
images=glob("train/**/**")
for i in range(9):
    plotnumber=331
    image=random.choice(images)
    plt.figure(figsize=(12,12))
    plt.subplot(plotnumber)
    plotnumber+=1
    plt.imshow(cv2.imread(image))

**prepair data For Training**

In [ ]:
img_size=48
batch_size=64
datagen_train=ImageDataGenerator()
validation_generator=datagen_train.flow_from_directory("train/",color_mode="grayscale",target_size=(img_size,img_size),batch_size=batch_size,class_mode="categorical",shuffle=True)
datagen_validation=ImageDataGenerator()
validation_generator=datagen_train.flow_from_directory("test/",color_mode="grayscale",target_size=(img_size,img_size),batch_size=batch_size,class_mode="categorical",shuffle=True)


**Defining model**

In [ ]:
def convolution(input_tensor,filters,kernal_size):
    x=conv2D(filters=filters,kernal_size=kernal_size,padding="same")(input_tensor)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    x=MaxPooling2D(pool_size=(2,2))(x)
    x=Dropout(0.25)(x)
    return x

In [ ]:
def denser_f(input_tensor,nodes):
    x=Denser(nodes)(input_tensor)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    x=Dropout(0.25)(x)
    return x

In [ ]:
def model_f(input_shape):
    inputs=input(input_shape)
    conv1=convolution(inputs,32,(3,3))
    conv2=convolution(inputs,64,(5,5))
    conv3=convolution(inputs,128,(3,3))
    
    flatten=Flatten()(conv3)
    dens_1=denser_f(flatten,256)
    output=Dense(7,activation="softmax")(dens_1)
    model=Model(inputs=[inputs],outputs=[output])
    model.compile(loss=["categorical_crossentropy"],optimizer="adam",metrics=["accuracy"])
    
    return model

In [ ]:
model1=model_f((48,48,1))
model1.summary()

**initializing Model**

In [ ]:
epochs=15
steps_per_epochs=train_generator.n//train_generator.batch_size
validation_steps=validation_generator.n//validation_generator.batch_size
checkpoint=ModelCheckpoint("model_weights.h5",monito="val_accuracy",save_weights_only=True,mode="max",verbose=1)
callbacks=[checkpoint]

**Training the Model**

In [ ]:
histroy=model.fit(
x=train_generator,
    steps_for_epoch=steps_for_epoch,
    epoch=epoch,
    validation_data=validation_generator
    validation_steps=validation_steps,
    callbacks=callbacks
)


**Evaluation**

In [ ]:
model.evaluate(validation_generator)

**loss**

In [ ]:
plt.plot(history.History["loss"])
plt.plot(history.History["val_loss"])
plt.title("Model Loss")
plt.legend(["Train","Validation"],loc="upper left")
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0.0,hspace=0.25,wspace=0.35)

In [ ]:
plt.plot(history.History["accuracy"])
plt.plot(history.History["val_accuracy"])
plt.title("Model Loss")
plt.legend(["Train","Validation"],loc="upper left")
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0.0,hspace=0.25,wspace=0.35)

In [ ]:
model_json=model.to_json()
with open("model_data","w") as file:
    file.write(model_json)